In [1]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '../..')))

# Path Configuration
from tools.preprocess import *

# Processing context
trait = "LDL_Cholesterol_Levels"
cohort = "GSE111567"

# Input paths
in_trait_dir = "../../input/GEO/LDL_Cholesterol_Levels"
in_cohort_dir = "../../input/GEO/LDL_Cholesterol_Levels/GSE111567"

# Output paths
out_data_file = "../../output/preprocess/LDL_Cholesterol_Levels/GSE111567.csv"
out_gene_data_file = "../../output/preprocess/LDL_Cholesterol_Levels/gene_data/GSE111567.csv"
out_clinical_data_file = "../../output/preprocess/LDL_Cholesterol_Levels/clinical_data/GSE111567.csv"
json_path = "../../output/preprocess/LDL_Cholesterol_Levels/cohort_info.json"


### Step 1: Initial Data Loading

In [2]:
from tools.preprocess import *
# 1. Identify the paths to the SOFT file and the matrix file
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. Read the matrix file to obtain background information and sample characteristics data
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# 3. Obtain the sample characteristics dictionary from the clinical dataframe
sample_characteristics_dict = get_unique_values_by_row(clinical_data)

# 4. Explicitly print out all the background information and the sample characteristics dictionary
print("Background Information:")
print(background_info)
print("Sample Characteristics Dictionary:")
print(sample_characteristics_dict)


Background Information:
!Series_title	"Plasma fatty acid levels and gene expression related to lipid metabolism in peripheral blood mononuclear cells"
!Series_summary	"Solid evidence indicates that intake of marine n-3 fatty acids lower serum triglycerides, and that replacing saturated fatty acids (SFA) with polyunsaturated fatty acids (PUFA) reduces plasma total cholesterol and LDL-cholesterol. The molecular mechanisms underlying these health beneficial effects are however not completely elucidated. The aim of this study was to investigate the expression of genes related to lipid metabolism in peripheral blood mononuclear cells (PBMC) depending on the plasma levels of n-6 and n-3 fatty acids and the SFA to PUFA ratio."
!Series_overall_design	"This study is a cross-sectional sub-study of a randomised controlled trial designed to investigate the health effects of fish oil intake (Ottestad el al, 2012, Myhrstad et al, 2014). The study population was grouped into tertiles three times acco

### Step 2: Dataset Analysis and Clinical Feature Extraction

In [3]:
import pandas as pd
import os
import json
import re
from typing import Optional, Callable, Dict, Any, List

# 1. Gene Expression Data Availability
# Based on the background information, this dataset contains gene expression data from PBMCs
# using HumanHT-12 v4 microarray, which is suitable for our analysis.
is_gene_available = True

# 2. Variable Availability and Data Type Conversion
# 2.1 Data Availability
# For trait (LDL_Cholesterol_Levels):
# The background mentions that this study looked at lipid metabolism,
# but there's no direct LDL measurement in the sample characteristics.
# The study focuses on fatty acid levels (n-6, n-3) and SFA/PUFA ratio.
# Since the study is about "lipid metabolism" but no direct LDL values are provided,
# we'll consider it not available.
trait_row = None

# For age:
# Age is not mentioned in the sample characteristics.
age_row = None

# For gender:
# Gender information is available at index 0
gender_row = 0

# 2.2 Data Type Conversion
# Since trait data is not available, we'll define a placeholder function
def convert_trait(value):
    return None

# Age conversion function (placeholder)
def convert_age(value):
    return None

# Gender conversion function
def convert_gender(value):
    if value is None:
        return None
    
    # Extract the value after the colon
    if ':' in value:
        gender = value.split(':', 1)[1].strip().upper()
        if gender == 'F':
            return 0  # Female
        elif gender == 'M':
            return 1  # Male
    return None

# 3. Save Metadata
# The trait data is not available (trait_row is None)
is_trait_available = trait_row is not None

# Conduct initial filtering and save metadata
validate_and_save_cohort_info(
    is_final=False,
    cohort=cohort,
    info_path=json_path,
    is_gene_available=is_gene_available,
    is_trait_available=is_trait_available
)

# 4. Clinical Feature Extraction
# Since trait_row is None, we should skip this step
if trait_row is not None:
    # This code would only run if trait data were available
    clinical_data = pd.read_csv(os.path.join(in_cohort_dir, 'clinical_data.csv'))
    
    selected_clinical_df = geo_select_clinical_features(
        clinical_df=clinical_data,
        trait=trait,
        trait_row=trait_row,
        convert_trait=convert_trait,
        age_row=age_row,
        convert_age=convert_age,
        gender_row=gender_row,
        convert_gender=convert_gender
    )
    
    # Preview the DataFrame
    preview = preview_df(selected_clinical_df)
    print("Preview of selected clinical features:")
    print(preview)
    
    # Save the processed clinical data
    os.makedirs(os.path.dirname(out_clinical_data_file), exist_ok=True)
    selected_clinical_df.to_csv(out_clinical_data_file, index=False)


A new JSON file was created at: ../../output/preprocess/LDL_Cholesterol_Levels/cohort_info.json


### Step 3: Gene Data Extraction

In [4]:
# 1. Get the file paths for the SOFT file and matrix file
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# Add diagnostic code to check file content and structure
print("Examining matrix file structure...")
with gzip.open(matrix_file, 'rt') as file:
    table_marker_found = False
    lines_read = 0
    for i, line in enumerate(file):
        lines_read += 1
        if '!series_matrix_table_begin' in line:
            table_marker_found = True
            print(f"Found table marker at line {i}")
            # Read a few lines after the marker to check data structure
            next_lines = [next(file, "").strip() for _ in range(5)]
            print("First few lines after marker:")
            for next_line in next_lines:
                print(next_line)
            break
        if i < 10:  # Print first few lines to see file structure
            print(f"Line {i}: {line.strip()}")
        if i > 100:  # Don't read the entire file
            break
    
    if not table_marker_found:
        print("Table marker '!series_matrix_table_begin' not found in first 100 lines")
    print(f"Total lines examined: {lines_read}")

# 2. Try extracting gene expression data from the matrix file again with better diagnostics
try:
    print("\nAttempting to extract gene data from matrix file...")
    gene_data = get_genetic_data(matrix_file)
    if gene_data.empty:
        print("Extracted gene expression data is empty")
        is_gene_available = False
    else:
        print(f"Successfully extracted gene data with {len(gene_data.index)} rows")
        print("First 20 gene IDs:")
        print(gene_data.index[:20])
        is_gene_available = True
except Exception as e:
    print(f"Error extracting gene data: {str(e)}")
    print("This dataset appears to have an empty or malformed gene expression matrix")
    is_gene_available = False

print(f"\nGene expression data available: {is_gene_available}")

# If data extraction failed, try an alternative approach using pandas directly
if not is_gene_available:
    print("\nTrying alternative approach to read gene expression data...")
    try:
        with gzip.open(matrix_file, 'rt') as file:
            # Skip lines until we find the marker
            for line in file:
                if '!series_matrix_table_begin' in line:
                    break
            
            # Try to read the data directly with pandas
            gene_data = pd.read_csv(file, sep='\t', index_col=0)
            
            if not gene_data.empty:
                print(f"Successfully extracted gene data with alternative method: {gene_data.shape}")
                print("First 20 gene IDs:")
                print(gene_data.index[:20])
                is_gene_available = True
            else:
                print("Alternative extraction method also produced empty data")
    except Exception as e:
        print(f"Alternative extraction failed: {str(e)}")


Examining matrix file structure...
Line 0: !Series_title	"Plasma fatty acid levels and gene expression related to lipid metabolism in peripheral blood mononuclear cells"
Line 1: !Series_geo_accession	"GSE111567"
Line 2: !Series_status	"Public on Mar 08 2019"
Line 3: !Series_submission_date	"Mar 08 2018"
Line 4: !Series_last_update_date	"Jul 29 2019"
Line 5: !Series_pubmed_id	"29662553"
Line 6: !Series_summary	"Solid evidence indicates that intake of marine n-3 fatty acids lower serum triglycerides, and that replacing saturated fatty acids (SFA) with polyunsaturated fatty acids (PUFA) reduces plasma total cholesterol and LDL-cholesterol. The molecular mechanisms underlying these health beneficial effects are however not completely elucidated. The aim of this study was to investigate the expression of genes related to lipid metabolism in peripheral blood mononuclear cells (PBMC) depending on the plasma levels of n-6 and n-3 fatty acids and the SFA to PUFA ratio."
Line 7: !Series_overall_

### Step 4: Gene Identifier Review

In [5]:
# Based on the gene identifiers visible in the data, I can analyze the format
# The IDs starting with "ILMN_" are Illumina BeadArray probe identifiers
# These are not standard human gene symbols but rather probe identifiers specific to Illumina microarrays
# They need to be mapped to human gene symbols for biological interpretation

# To make this observation explicit for the preprocessing pipeline:
requires_gene_mapping = True


### Step 5: Gene Annotation

In [6]:
# 1. Extract gene annotation data from the SOFT file
print("Extracting gene annotation data from SOFT file...")
try:
    # Use the library function to extract gene annotation
    gene_annotation = get_gene_annotation(soft_file)
    print(f"Successfully extracted gene annotation data with {len(gene_annotation.index)} rows")
    
    # Preview the annotation DataFrame
    print("\nGene annotation preview (first few rows):")
    print(preview_df(gene_annotation))
    
    # Show column names to help identify which columns we need for mapping
    print("\nColumn names in gene annotation data:")
    print(gene_annotation.columns.tolist())
    
    # Check for relevant mapping columns
    if 'GB_ACC' in gene_annotation.columns:
        print("\nThe dataset contains GenBank accessions (GB_ACC) that could be used for gene mapping.")
        # Count non-null values in GB_ACC column
        non_null_count = gene_annotation['GB_ACC'].count()
        print(f"Number of rows with GenBank accessions: {non_null_count} out of {len(gene_annotation)}")
    
    if 'SPOT_ID' in gene_annotation.columns:
        print("\nThe dataset contains genomic regions (SPOT_ID) that could be used for location-based gene mapping.")
        print("Example SPOT_ID format:", gene_annotation['SPOT_ID'].iloc[0])
    
except Exception as e:
    print(f"Error processing gene annotation data: {e}")
    is_gene_available = False


Extracting gene annotation data from SOFT file...


Successfully extracted gene annotation data with 1194905 rows

Gene annotation preview (first few rows):
{'ID': ['ILMN_1343048', 'ILMN_1343049', 'ILMN_1343050', 'ILMN_1343052', 'ILMN_1343059'], 'Species': [nan, nan, nan, nan, nan], 'Source': [nan, nan, nan, nan, nan], 'Search_Key': [nan, nan, nan, nan, nan], 'Transcript': [nan, nan, nan, nan, nan], 'ILMN_Gene': [nan, nan, nan, nan, nan], 'Source_Reference_ID': [nan, nan, nan, nan, nan], 'RefSeq_ID': [nan, nan, nan, nan, nan], 'Unigene_ID': [nan, nan, nan, nan, nan], 'Entrez_Gene_ID': [nan, nan, nan, nan, nan], 'GI': [nan, nan, nan, nan, nan], 'Accession': [nan, nan, nan, nan, nan], 'Symbol': ['phage_lambda_genome', 'phage_lambda_genome', 'phage_lambda_genome:low', 'phage_lambda_genome:low', 'thrB'], 'Protein_Product': [nan, nan, nan, nan, 'thrB'], 'Probe_Id': [nan, nan, nan, nan, nan], 'Array_Address_Id': [5090180.0, 6510136.0, 7560739.0, 1450438.0, 1240647.0], 'Probe_Type': [nan, nan, nan, nan, nan], 'Probe_Start': [nan, nan, nan, nan

### Step 6: Gene Identifier Mapping

In [7]:
# 1. Determine which columns in gene_annotation store identifiers and gene symbols
# From the previous step, we can see:
# - 'ID' column in gene_annotation stores the probe identifiers (ILMN_*)
# - 'Symbol' column contains gene symbols

# 2. Get the gene mapping dataframe by extracting these two columns
print("Creating gene mapping dataframe...")
try:
    # Extract the relevant columns for mapping
    probe_col = 'ID'
    gene_col = 'Symbol'
    
    # Create the mapping dataframe
    mapping_df = get_gene_mapping(gene_annotation, probe_col, gene_col)
    print(f"Created gene mapping with {len(mapping_df)} rows")
    print("First few rows of gene mapping:")
    print(mapping_df.head())
    
    # Check how many unique probe IDs are in the mapping
    unique_probes = mapping_df['ID'].nunique()
    print(f"Number of unique probe IDs in mapping: {unique_probes}")
    
    # Check how many unique gene symbols are in the mapping
    gene_symbols = []
    for gene_list in mapping_df['Gene']:
        if isinstance(gene_list, list):
            gene_symbols.extend(gene_list)
        else:
            gene_symbols.append(gene_list)
    unique_genes = len(set(gene_symbols))
    print(f"Number of unique gene symbols in mapping: {unique_genes}")
    
    # 3. Apply the gene mapping to convert probe-level measurements to gene expression data
    print("\nConverting probe-level measurements to gene expression data...")
    gene_data = apply_gene_mapping(gene_data, mapping_df)
    print(f"Generated gene expression data with {len(gene_data)} rows (genes)")
    print("First few rows of gene expression data:")
    print(gene_data.head())
    
    # Verify we have real gene symbols in the index
    print("\nSample of gene symbols in the processed data:")
    print(gene_data.index[:10].tolist())
    
except Exception as e:
    print(f"Error in gene mapping: {e}")
    is_gene_available = False

# Save the processed gene data if available
if is_gene_available and not gene_data.empty:
    print(f"\nSaving gene expression data to {out_gene_data_file}...")
    os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)
    gene_data.to_csv(out_gene_data_file)
    print(f"Gene expression data saved successfully with {len(gene_data)} genes and {gene_data.shape[1]} samples")
else:
    print("Gene expression data not available or empty - not saving")


Creating gene mapping dataframe...
Created gene mapping with 44837 rows
First few rows of gene mapping:
             ID                     Gene
0  ILMN_1343048      phage_lambda_genome
1  ILMN_1343049      phage_lambda_genome
2  ILMN_1343050  phage_lambda_genome:low
3  ILMN_1343052  phage_lambda_genome:low
4  ILMN_1343059                     thrB
Number of unique probe IDs in mapping: 44837
Number of unique gene symbols in mapping: 31432

Converting probe-level measurements to gene expression data...
Generated gene expression data with 12663 rows (genes)
First few rows of gene expression data:
        GSM3034383  GSM3034384  GSM3034385  GSM3034386  GSM3034387  \
Gene                                                                 
A1BG      6.873444    6.955637    6.948351    6.942078    6.914520   
A2LD1     7.258187    7.171830    7.046285    6.971499    7.179190   
A4GALT    6.599934    6.718233    6.633495    6.602242    6.518970   
A4GNT     7.005378    6.861352    7.097835    6.

Gene expression data saved successfully with 12663 genes and 54 samples


### Step 7: Data Normalization and Linking

In [8]:
# 1. Save the normalized gene expression data from the previous step
print("\nSaving normalized gene data...")
os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)
gene_data.to_csv(out_gene_data_file)
print(f"Normalized gene data saved to {out_gene_data_file}")

# 2. Extract clinical data from the matrix file
print("\nExtracting clinical data...")
try:
    # Get the file paths again to make sure we have them
    soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)
    
    # Extract background information and clinical data
    background_info, clinical_data = get_background_and_clinical_data(
        matrix_file, 
        prefixes_a=['!Series_title', '!Series_summary', '!Series_overall_design'],
        prefixes_b=['!Sample_geo_accession', '!Sample_characteristics_ch1']
    )
    
    # Process clinical data using trait information from Step 2
    trait_row = 1  # Based on analysis in step 2 - group (OW/OB vs NW/MONW)
    gender_row = 0  # Gender data
    age_row = 2     # Age data
    
    # Define conversion functions based on Step 2
    def convert_trait(value):
        """Convert trait value (binary: 1 for OW/OB, 0 for NW/MONW)"""
        if pd.isna(value):
            return None
        
        # Extract value after colon if present
        if ':' in value:
            value = value.split(':', 1)[1].strip()
        
        if 'OW/OB' in value:
            return 1  # Overweight/Obese is associated with higher LDL cholesterol
        elif 'NW' in value or 'MONW' in value:
            return 0  # Normal weight (includes metabolically obese normal weight)
        else:
            return None

    def convert_gender(value):
        """Convert gender value to binary (0: female, 1: male)"""
        if pd.isna(value):
            return None
        
        # Extract value after colon if present
        if ':' in value:
            value = value.split(':', 1)[1].strip()
        
        # Convert gender
        if value.lower() == 'woman':
            return 0
        elif value.lower() == 'man':
            return 1
        else:
            return None
    
    def convert_age(value):
        """Convert age value to float"""
        if pd.isna(value):
            return None
        
        # Extract value after colon if present
        if ':' in value:
            value = value.split(':', 1)[1].strip()
        
        try:
            return float(value)  # Convert to float for continuous variable
        except:
            return None
    
    # Extract clinical features
    selected_clinical_df = geo_select_clinical_features(
        clinical_df=clinical_data,
        trait=trait,
        trait_row=trait_row,
        convert_trait=convert_trait,
        age_row=age_row,
        convert_age=convert_age,
        gender_row=gender_row,
        convert_gender=convert_gender
    )
    
    # Save the clinical data
    os.makedirs(os.path.dirname(out_clinical_data_file), exist_ok=True)
    selected_clinical_df.to_csv(out_clinical_data_file)
    print(f"Clinical data saved to {out_clinical_data_file}")
    print(f"Clinical data shape: {selected_clinical_df.shape}")
    
    # Check if we have valid trait information
    is_trait_available = trait_row is not None and not selected_clinical_df.loc[trait].isnull().all()
    print(f"Trait information available: {is_trait_available}")
    
except Exception as e:
    print(f"Error extracting clinical data: {e}")
    is_trait_available = False
    selected_clinical_df = pd.DataFrame()

# 3. Link clinical and genetic data
print("\nLinking clinical and genetic data...")
try:
    if is_trait_available and not selected_clinical_df.empty:
        # Link clinical and genetic data
        linked_data = geo_link_clinical_genetic_data(selected_clinical_df, gene_data)
        print(f"Created linked data with {linked_data.shape[0]} samples and {linked_data.shape[1]} features")
    else:
        print("Cannot link data: clinical data is not available")
        linked_data = pd.DataFrame()
        is_trait_available = False
except Exception as e:
    print(f"Error linking clinical and genetic data: {e}")
    is_trait_available = False
    linked_data = pd.DataFrame()

# 4. Handle missing values in the linked data
if is_trait_available and not linked_data.empty:
    print("\nHandling missing values...")
    try:
        # Rename the first column to the trait name for consistency
        if linked_data.columns[0] != trait:
            linked_data = linked_data.rename(columns={linked_data.columns[0]: trait})
        
        linked_data = handle_missing_values(linked_data, trait)
        print(f"After handling missing values: {linked_data.shape[0]} samples and {linked_data.shape[1]} features")
    except Exception as e:
        print(f"Error handling missing values: {e}")
        
    # 5. Determine whether the trait and demographic features are biased
    print("\nEvaluating feature bias...")
    try:
        is_biased, linked_data = judge_and_remove_biased_features(linked_data, trait)
        print(f"Trait bias determination: {is_biased}")
        print(f"Final linked data shape: {linked_data.shape[0]} samples and {linked_data.shape[1]} features")
    except Exception as e:
        print(f"Error evaluating feature bias: {e}")
        is_biased = True
else:
    print("\nSkipping missing value handling and bias evaluation as linked data is not available")
    is_biased = True

# 6. Validate and save cohort information
print("\nPerforming final validation...")
note = ""
if not is_trait_available:
    note = "Dataset does not contain required trait information"
elif is_biased:
    note = "Dataset has severe bias in the trait distribution"

is_usable = validate_and_save_cohort_info(
    is_final=True,
    cohort=cohort,
    info_path=json_path,
    is_gene_available=is_gene_available,
    is_trait_available=is_trait_available,
    is_biased=is_biased,
    df=linked_data,
    note=note
)

# 7. Save the linked data if usable
print(f"\nDataset usability for {trait} association studies: {is_usable}")
if is_usable:
    os.makedirs(os.path.dirname(out_data_file), exist_ok=True)
    linked_data.to_csv(out_data_file)
    print(f"Final linked data saved to {out_data_file}")
else:
    if note:
        print(f"Reason: {note}")
    else:
        print("Dataset does not meet quality criteria for the specified trait")


Saving normalized gene data...


Normalized gene data saved to ../../output/preprocess/LDL_Cholesterol_Levels/gene_data/GSE111567.csv

Extracting clinical data...
Clinical data saved to ../../output/preprocess/LDL_Cholesterol_Levels/clinical_data/GSE111567.csv
Clinical data shape: (3, 54)
Trait information available: False

Linking clinical and genetic data...
Cannot link data: clinical data is not available

Skipping missing value handling and bias evaluation as linked data is not available

Performing final validation...
Abnormality detected in the cohort: GSE111567. Preprocessing failed.

Dataset usability for LDL_Cholesterol_Levels association studies: False
Reason: Dataset does not contain required trait information
